In [1]:
# market_comparison.ipynb
import sys
import os
sys.path.append('..')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Import our analysis modules
import metrics
import visualize
import compare

# Set plotting style for publication-quality figures
plt.style.use('seaborn-whitegrid')
sns.set_context("paper", font_scale=1.5)
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.family'] = 'serif'

# Calculate metrics for all scenarios
results = {}
for scen in range(1, 6):
    try:
        results[scen] = metrics.calculate_all_metrics(scen, "ref")
    except Exception as e:
        print(f"Error processing Scenario {scen}: {e}")

# Create summary table
summary_df = compare.create_scenario_summary_table(results)
summary_df

# Load data for all scenarios
scenario_data = visualize.load_all_scenarios()

# Create price duration curves for thesis
plt.figure(figsize=(10, 6))
for scen, df in scenario_data.items():
    if scen in [2, 4, 5]:  # Compare key scenarios
        prices = df["Price"].sort_values(ascending=False).values
        plt.plot(range(len(prices)), prices, linewidth=2, 
                 label=compare.scenario_names().get(scen, f"Scenario {scen}"))

plt.title("Price Duration Curves", fontsize=16)
plt.xlabel("Hours (sorted by price)", fontsize=14)
plt.ylabel("Price (€/MWh)", fontsize=14)
plt.grid(True, alpha=0.3)
plt.legend(fontsize=12)
plt.tight_layout()
plt.savefig("thesis_figures/price_duration_comparison.png")
plt.show()

# Capacity market comparison
cm_scenarios = [s for s in results.keys() if s in [4, 5]]
if len(cm_scenarios) >= 2:
    # Compare generator revenues
    gen_comparison = compare.create_generator_comparison(results, cm_scenarios)
    
    # Plot comparison
    gen_types = gen_comparison["Generator"]
    
    # Prepare data for plotting
    s4_energy = gen_comparison.filter(like="Scenario 4 - Energy").values.flatten()
    s4_capacity = gen_comparison.filter(like="Scenario 4 - Capacity").values.flatten()
    s5_energy = gen_comparison.filter(like="Scenario 5 - Energy").values.flatten()
    s5_capacity = gen_comparison.filter(like="Scenario 5 - Capacity").values.flatten()
    
    # Create plot
    fig, ax = plt.subplots(figsize=(12, 8))
    width = 0.35
    x = np.arange(len(gen_types))
    
    # Plot bars
    ax.bar(x - width/2, s4_energy, width, label='Central CM - Energy Revenue')
    ax.bar(x - width/2, s4_capacity, width, bottom=s4_energy, label='Central CM - Capacity Revenue')
    ax.bar(x + width/2, s5_energy, width, label='Decentral CM - Energy Revenue')
    ax.bar(x + width/2, s5_capacity, width, bottom=s5_energy, label='Decentral CM - Capacity Revenue')
    
    # Add labels
    ax.set_xlabel('Generator Type', fontsize=14)
    ax.set_ylabel('Revenue (M€)', fontsize=14)
    ax.set_title('Generator Revenue: Central vs. Decentralized CM', fontsize=16)
    ax.set_xticks(x)
    ax.set_xticklabels(gen_types)
    ax.legend()
    
    plt.tight_layout()
    plt.savefig("thesis_figures/central_vs_decentral_revenue.png")
    plt.show()

# Reliability metrics comparison
reliability_df = pd.DataFrame({
    'Scenario': [compare.scenario_names().get(s) for s in results.keys()],
    'LOLE (hours)': [results[s]['reliability_metrics']['LOLE'] for s in results.keys()],
    'ENS 95th percentile (GWh)': [results[s]['reliability_metrics']['ENS_95'] for s in results.keys()]
})

fig, ax = plt.subplots(figsize=(10, 6))
reliability_df.plot(x='Scenario', y=['LOLE (hours)'], kind='bar', ax=ax)
ax.set_ylabel('Hours per year')
ax.set_title('Loss of Load Expectation')
plt.tight_layout()
plt.savefig("thesis_figures/reliability_comparison.png")
plt.show()

# Export key results for thesis tables
summary_df.to_csv("thesis_tables/scenario_summary.csv")

# Generate comparative visualization of consumer costs
compare.create_consumer_comparison(results).to_csv("thesis_tables/consumer_cost_breakdown.csv")

# Create summary dashboard for presentation
visualize.create_summary_dashboard(results, save_path="thesis_figures/dashboard.png")

ArgumentError: ArgumentError: Package sys not found in current path.
- Run `import Pkg; Pkg.add("sys")` to install the sys package.